In [1]:
import os
import numpy as np
import pickle
import cv2
import os
import csv
import math
import pandas as pd
import matplotlib.pyplot as plt
import skimage.segmentation as seg
import skimage.transform as tran
from skimage import io, filters, transform, color, measure, segmentation, morphology, feature,img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
import matplotlib.patches as mpatches

In [2]:
def image_segmentation(img):
    
    ##knn
    label_image = segmentation.slic(img,n_segments = 7)
    image_label_overlay = color.label2rgb(label_image,image = img)

    ##tresh
    #image = color.rgb2gray(img)
    #thresh =filters.threshold_otsu(image)
    #bw =morphology.closing(image > thresh, morphology.square(3))
    
    #cleared = bw.copy() 
    #segmentation.clear_border(cleared) 
    #label_image =measure.label(cleared) 
    #borders = np.logical_xor(bw, cleared)
    #label_image[borders] = -1
    #image_label_overlay =color.label2rgb(label_image, image=image)
    #fig,(ax0,ax1)= plt.subplots(1,2, figsize=(8, 6))

    #ax0.imshow(cleared,plt.cm.gray)
    #ax1.imshow(image_label_overlay)
    
    MR1 = 0
    MR2 = 0
    MC1 = 0
    MC2 = 0
    A,B = img.shape[:2]
    dis = max(A,B)
    a = A/2
    b = B/2

    for region in measure.regionprops(label_image):

        if region.area < 400:
            continue

        minr, minc, maxr, maxc = region.bbox
        d = (maxc - minc)/2 + minc
        c = (maxr - minr)/2 + minr
        D = distance(a,b,c,d)
        #rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
        #                          fill=False, edgecolor='red', linewidth=2)
        #ax1.add_patch(rect)
        if D < dis:
        #if (abs(maxc - minc)* abs(maxr - minr)) > (abs(MC2-MC1)* abs(MR2-MR1)):
            MR1 = minr
            MR2 = maxr
            MC1 = minc
            MC2 = maxc
            dis = D

    #fig.tight_layout()
    #plt.show()
    
    #roi = color.rgb2gray(img)
    #thresh = 140
    #bw = (roi <= thresh/255) * 1
    #pic_temp2 = imfill(bw)
    #cleared = pic_temp2.copy()  
    #segmentation.clear_border(cleared)
    #label_image = measure.label(cleared) 
    #borders = np.logical_xor(bw, cleared)  
    #label_image[borders] = -1
    #Eccentricity = 1   
    #for region in measure.regionprops(label_image):
    #   if region.area < 100:
    #        continue
    #    print('area is ' + str(region.area) + '  ecc is' + str(region.eccentricity))
    #    if Eccentricity > region.eccentricity:
    #        Eccentricity = region.eccentricity
    #        minr, minc, maxr, maxc = region.bbox 
    
    if 'MR1' in vars():
        pic = img[MR1:MR2, MC1:MC2,:]
        pic = transform.resize(pic, [256,256,3])
        #print(lbp)
        #plt.imshow(pic)
        #plt.show()
    else:
        pic = transform.resize(img, [256, 256, 3])
        
    return pic

In [3]:
def distance(a,b,c,d):
    p1=np.array([a,b])
    p2=np.array([c,d])
    p3=p2-p1
    p4=math.hypot(p3[0],p3[1])
    
    return p4

In [4]:
def imfill(im_th):
    
    # Copy the thresholded imapge.
    im_th = np.uint8(im_th)
    im_floodfill = im_th.copy()
    # Mask used to flood filling.
    # Notice the size needs to be 2 pixels than the image.
    h, w = im_th.shape[:2]
    # print('h '+str(h)+'   w '+str(w))
    mask = np.zeros((h + 2, w + 2), np.uint8)
    # Floodfill from point (0, 0)
    cv2.floodFill(im_floodfill, mask, (0, 0), 1)
    # Invert floodfilled image
    im_floodfill_inv = cv2.bitwise_not(im_floodfill)
    # Combine the two images to get the foreground.
    im_out = im_th | im_floodfill_inv
    return im_out

In [ ]:
X = [] #image paths
Y = [] #image labels

n = 8
for i in range(1,n+1):
    for f in os.listdir('.\\photo\\%s' % i):
        X.append('.\\photo\\' +str(i) + '\\' + str(f))
        Y.append(i)

X = np.array(X)
Y = np.array(Y)  

In [5]:
def LBP_feature(I):
    pic1 = color.rgb2gray(I)
    rows, cols = pic1.shape
    radius = 4;
    n_points = radius * 8
    lbp_sum=[]
    for row in range(radius):
        for col in range(radius):
            #print(str((row * rows//2)) + ' : ' + str(((row+1) * rows//2 - 1)))
            pic1_block = pic1[(row * rows//2) : ((row+1) * rows//2 - 1) , (col * col//2) : ((col+1) * col//2 - 1)]
            lbp = feature.local_binary_pattern(pic1, n_points, radius, 'uniform')
            lbp2 = lbp.astype(np.int32)
            max_bins = int(lbp2.max() + 1)
            train_hist, _ = np.histogram(lbp2, normed = True, bins=max_bins, range=(0, max_bins))
           # print(train_hist.dtype)
            #print(train_hist)
            lbpfeature=lbp_sum + train_hist.tolist()
    return lbpfeature

In [6]:
def hist_feature(I):
    #img = cv2.cvtColor(np.array(I),cv2.COLOR_RGB2BGR)
    #r,g,b = cv2.split(I)
    #bgr_image = cv2.merge([b,g,r])
    #bgr_image = bgr_image*255
    hist = cv2.calcHist([I],[0,1],None,[256,256], [0.0,255.0,0.0,255.0])
    histfeature = (hist/255).flatten()
    return histfeature

In [ ]:
#Image segmentation

#image_show = []
i = 0
X_path = []

for path in X:
    if os.path.exists(path):
        img = io.imread(path)
        
        I = image_segmentation(img)
        io.imsave('./photo_seg/%s.png' %i,img_as_ubyte(I))
        X_path.append('./photo_seg/%s.png' %i)
        #image_show.append(I)


In [ ]:
#Feature extraction
X_lbp = []


for path in X_path:
    I = io.imread(path)
    feature1 = LBP_feature(I)
    X_lbp.append(feature1)
    
    print(path[12:])   

In [ ]:
X_hist = []
for path in X:
    I = cv2.imread(path)
    image = cv2.resize(I, (256,256),interpolation=cv2.INTER_CUBIC)
    feature2 = hist_feature(image)
    X_hist.append(feature2)
    
    print(path[12:]) 

In [ ]:
#FEATURE SAVE
LBP = open('LBP.pickle','wb')
pickle.dump(X_lbp,LBP)
LBP.close()

HIST = open('HIST.pickle','wb')
pickle.dump(X_hist,HIST)
HIST.close()

In [ ]:
#Feature read
LBP = open('LBP.pickle','rb')
X_lbp = pickle.load(LBP)
#print(X_lbp)

HIST = open('HIST.pickle','rb')
X_hist = pickle.load(HIST)
#print(X_hist)

In [ ]:
#lbp
X_lbp_train, X_lbp_test, y_lbp_train, y_lbp_test = train_test_split(X_lbp, Y,test_size=0.3, random_state=0)
#hist
X_hist_train, X_hist_test, y_hist_train, y_hist_test = train_test_split(X_hist, Y,test_size=0.3, random_state=0)

In [ ]:
from sklearn import svm, multiclass, model_selection
svr_rbf = svm.SVR(kernel='rbf', C=1e3, gamma=0.1);
model = multiclass .OneVsRestClassifier(svr_rbf,-1)  #.fit(train_data, train_label).score(test_data,test_label)
clf_lbp = model.fit(X_lbp_train, y_lbp_train)
predictions_labels_lbp = clf_lbp.predict(X_lbp_test)

print ('Evaluation:')
print (classification_report(y_lbp_test, predictions_labels_lbp))

print('Score:')
sore_lbp=clf_lbp.score(X_lbp_test, y_lbp_test)
print(sore_lbp)

In [ ]:
clf_hist = BernoulliNB().fit(X_hist_train, y_hist_train)
predictions_labels_hist = clf_hist.predict(X_hist_test)

#print ('Prediction:')
#print (predictions_labels_hist)

print ('Evaluation:')
print (classification_report(y_hist_test, predictions_labels_hist))

print('Score:')
score_hist = clf_hist.score(X_hist_test,y_hist_test)
print(score_hist)

In [ ]:
from sklearn.externals import joblib

joblib.dump(clf_hist, 'clf_hist.model')
joblib.dump(clf_lbp, 'clf_lbp.model')

In [ ]:
def network_parameter(Report_list,N):
    #n = len(Report_list)
    #M = N**n
    P = np.zeros(shape=(N,N**2))
    
    P_L = np.zeros(shape=(N,N))
    P_H = np.zeros(shape=(N,N))
    
    #P(O|L)
    for a in range(N):
        p = round(Report_list[0][str(a+1)]['precision'],2)
        for j in range(N):
            if j == a:
                P_L[j][a] = p
            else:
                P_L[j][a] = (1-p)/(N-1)
    print('parameter A:')
    print(P_L)
    
    #P(O=j|H=b)
    for b in range(N):
        p = round(Report_list[1][str(b+1)]['precision'],2)
        for j in range(N):
            if j == b:
                P_H[j][b] = p
            else:
                P_H[j][b] = (1-p)/(N-1)
    print('parameter B:')
    print(P_H)
    
    #P(O=j|L=a,H=b)=P(O=j|L=a)*P(O=j|H=b)
    for j in range(N):
        for b in range(N):
            for a in range(N):
                i = b * N + a
                #print(P.shape,i,j,a,b)
                P[j,i] = (P_L[a][j]*P_H[b][j])/(1/N)
    print('Conditional Probability Distribution:')
    print(P)
    
    return P,P_L,P_H

In [ ]:

LBP_predict = clf_lbp.predict(X_lbp_train)
Hist_predict = clf_hist.predict(X_hist_train)

LBP_test = clf_lbp.predict(X_lbp_test)
Hist_test = clf_hist.predict(X_hist_test)

In [ ]:
data =  {'lbp':LBP_predict,'hist':Hist_predict,'object':y_lbp_train}
values = pd.DataFrame(data)

test = {'lbp':LBP_test,'hist':Hist_test,'object':y_lbp_test}
test_values = pd.DataFrame(test)

true_object = pd.DataFrame({'object':y_lbp_test})

In [ ]:
test_values.drop('object', axis=1, inplace=True)

In [ ]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
#values = pd.DataFrame(np.random.randint(low=0, high=2, size=(1000, 5)),
#                      columns=['A', 'B', 'C', 'D', 'E'])
#train_data = values[:800]
#predict_data = values[800:]
model = BayesianModel([('lbp', 'object'), ('hist', 'object')])
model.fit(values)
predict_data = test_values.copy()
#test_values.drop('object', axis=1, inplace=True)
y_pred = model.predict(predict_data)

In [ ]:
y_pred.insert(1,'true',true_object['object'])

In [ ]:
num = 0
N = len(y_lbp_test)
for i in (np.array(y_pred['object'])-np.array(y_pred['true'])):
    if i != 0:
        print(i)
        num += 1
print(num,N)